In [1]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras_cv
import h5py

train_file = "isic-2024-challenge/train-metadata.csv"
df = pd.read_csv(train_file)

2024-08-20 06:13:31.410021: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using TensorFlow backend


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_677/1848604162.py:11: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(train_file)


In [2]:
df['isic_id'] = 'isic-2024-challenge/train-image/image/' + df['isic_id'] + '.jpg'

df_m = df[df['target'] == 1]
df_b = df[df['target'] == 0]

print(len(df_m.index))

df_b = df_b.sample(len(df_m.index))

df_balanced = pd.concat([df_m, df_b], axis=0)

#df_balanced['target'].value_counts().plot.pie()

393


In [3]:
train_df = df_balanced.sample(frac=0.8, random_state=42) 
  
# Dropping all those indexes from the dataframe that exists in the train_set 
test_df = df_balanced.drop(train_df.index) 


In [4]:

def process_data(dft):
    img_filenames = dft['isic_id']
    labels = dft['target']

    ds = tf.data.Dataset.from_tensor_slices((img_filenames, labels))

    def parse_image(filename, label):
        print(filename)   
        image = tf.io.read_file(filename)
        image = tf.io.decode_jpeg(image)
        img_shape = image.shape
        print("Shape is: ", img_shape)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image,
                                [256,256],
                                preserve_aspect_ratio=False,
                                antialias=False,
                                name=None
                                )
        return image, label
    
    ds = ds.map(parse_image)
    
    return ds


In [5]:
ds_train = process_data(train_df)
ds_val = process_data(test_df)

image, label  = next(iter(ds_train))

ds_train = ds_train.batch(32)
ds_val = ds_val.batch(32)

Tensor("args_0:0", shape=(), dtype=string)
Shape is:  (None, None, None)
Tensor("args_0:0", shape=(), dtype=string)
Shape is:  (None, None, None)


2024-08-20 06:13:38.687888: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-20 06:13:38.733531: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-20 06:13:38.734979: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
backbone = keras_cv.models.DenseNetBackbone.from_preset(
    "densenet201_imagenet"
)
model = keras_cv.models.ImageClassifier(
    backbone=backbone,
    num_classes=1,
    activation="relu",
)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)
model.summary()

model.fit(ds_train, validation_data=ds_val, epochs=10)

/usr/local/lib/python3.10/dist-packages/keras_cv/src/models/backbones/backbone.py:44: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_cv/src/models/backbones/backbone.py:44: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Model: "image_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 dense_net_backbone (DenseN  (None, None, None, 1920   18321984  
 etBackbone)                 )                                   
                                                                 
 avg_pool (GlobalAveragePoo  (None, 1920)              0         
 ling2D)                                                         
                                                                 
 predictions (Dense)         (None, 1)                 1921      
                                                                 
Total params: 18323905 (69.90 MB)
Trainable params: 18094849 (69.03 MB)
Non-trainable params: 229056 (894.75 KB)
_________________________________________________________________
Epo

2024-08-20 06:15:08.643482: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 90201
I0000 00:00:1724134512.054788     822 service.cc:145] XLA service 0x7869948a99a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724134512.054815     822 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-08-20 06:15:12.062284: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1724134512.132311     822 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/20 [==============================] - 117s 1s/step - loss: 2.5358 - accuracy: 0.5024 - val_loss: 7.6634 - val_accuracy: 0.5032
Epoch 2/10
20/20 [==============================] - 4s 213ms/step - loss: 2.1985 - accuracy: 0.5119 - val_loss: 7.6634 - val_accuracy: 0.5032
Epoch 3/10
20/20 [==============================] - 4s 210ms/step - loss: 1.8259 - accuracy: 0.5135 - val_loss: 0.9327 - val_accuracy: 0.5032
Epoch 4/10
20/20 [==============================] - 4s 212ms/step - loss: 1.6243 - accuracy: 0.5183 - val_loss: 0.7358 - val_accuracy: 0.5032
Epoch 5/10
20/20 [==============================] - 4s 203ms/step - loss: 1.4893 - accuracy: 0.5278 - val_loss: 0.7094 - val_accuracy: 0.5032
Epoch 6/10
20/20 [==============================] - 4s 201ms/step - loss: 1.3132 - accuracy: 0.5246 - val_loss: 0.7352 - val_accuracy: 0.5032
Epoch 7/10
20/20 [==============================] - 4s 204ms/step - loss: 1.2571 - accuracy: 0.5278 - val_loss: 0.7070 - val_accuracy: 0.5032
Epoch 8/10
20/20 [

In [7]:
hdf5_path = "isic-2024-challenge/test-image.hdf5"

results = []

with h5py.File(hdf5_path, 'r') as hf:
    for isic_id in hf.keys():
        jpeg_string = hf[isic_id][()]
        image = tf.io.decode_jpeg(jpeg_string)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image,
                                [256,256],
                                preserve_aspect_ratio=False,
                                antialias=False,
                                name=None
                                )                
        target = model.predict(np.array([image]), batch_size=1, steps=1)
        results.append([isic_id, target[0][0]])
    
submission_df = pd.DataFrame(results, columns=["isic_id", "target"])
submission_df.head()

1/1 [==============================] - 0s 27ms/step


,isic_id,target
0,ISIC_0015657,0.457660
1,ISIC_0015729,0.472336
2,ISIC_0015740,0.457515


In [8]:
submission_df.to_csv("submission.csv", index=False)